In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import re
import selenium
from pandas import DataFrame
import json

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
browser = init_browser()
url = "https://www.fragrantica.com/search/"
browser.visit(url)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\jvolo\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [24]:
# Start over whenever you change years or after error troubleshooting 
html = browser.html
soup = bs(html, "html.parser")

In [4]:
#list_of_perfume_dicts = []

In [25]:
html = browser.html
soup = bs(html, "html.parser")

for i in range(1944,1945):
    #try:
        ####### CHANGE THE YEAR SETTING #######
        browser.find_by_css('input[type="number"]')[1].fill(i)
        browser.find_by_css('input[type="number"]')[0].fill(i)
        time.sleep(5)

        ####### CLICK "SHOW MORE RESULTS" #######
        ####### with this coded out, we only get 40 results per year ######
        for j in range(0,100):
            try:
                browser.find_by_css('button[class="button"]').click()
            except:
                break
            
        html = browser.html
        soup = bs(html, "html.parser")

        ####### GET THE LENGTH OF THE RESULTS #######
        block1 = soup.find("span", class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")
        perfume_name_list = []
        
        for perf_text in block1.find_all("a", href=True):
            perf_name = perf_text.get_text()
            perfume_name_list.append(perf_name.replace('\n', '').strip(" "))

        ####### START ITERATING THROUGH THE PERFUMES OF THAT YEAR #######    
        for k in range(len(perfume_name_list)):
            html = browser.html
            soup = bs(html, "html.parser")

            browser.find_by_css('span[class="link-span"]')[k].click()

            time.sleep(5)

            html = browser.html
            soup = bs(html, "html.parser")

            time.sleep(5)

            ####### PULL FROM ORIGINAL NAME LIST #######
            perfume_name = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()
            perfume_comp = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()

            perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]

            for_gender = soup.find("small").get_text()
            
            try:
                rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
                number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
            except:
                rating = "NA"
                number_votes = "NA"
                print(f"{perfume_name} does not have a ranking")
                
            try:
                description = soup.find_all("div", class_="cell small-12")[3].get_text()
            except:
                description = "NA"
                print(f"{perfume_name} does not have a description")

            ####### MAIN ACCORDS DICTIONARY #######
            
            try:
                main_accords = soup.find_all("div", class_="cell accord-box")
                accords_dict = {}
                for m in range(len(main_accords)):
                    accord_name = main_accords[m].get_text()
                    accord_value = float(main_accords[m].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
                    accords_dict[accord_name] = accord_value
            except:
                accords_dict = {}
                print(f"{perfume_name} does not have accords")

            ####### FRAGRANCE NOTES #######        
            notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})

            if len(notes) == 3:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for n in range(len(notes[0].find_all("span", class_="link-span"))):
                    top_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for p in range(len(notes[1].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[1].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for q in range(len(notes[2].find_all("span", class_="link-span"))):
                    base_notes_list.append(notes[2].find_all("div")[number].get_text())
                    number += 3
            elif len(notes) == 2:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for r in range(len(notes[0].find_all("span", class_="link-span"))):
                    top_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3

                number = 2
                for s in range(len(notes[1].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[1].find_all("div")[number].get_text())
                    number += 3
            elif len(notes) == 1:
                number = 2
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

                for v in range(len(notes[0].find_all("span", class_="link-span"))):
                    middle_notes_list.append(notes[0].find_all("div")[number].get_text())
                    number += 3
            else:
                top_notes_list = []
                middle_notes_list = []
                base_notes_list = []

            ####### VOTING DATA & INFORMATION #######
            voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")

            ####### Longevity #######
            long_v_weak = int(voting[0].get_text())
            long_weak = int(voting[1].get_text())
            long_moderate = int(voting[2].get_text())
            long_long_last = int(voting[3].get_text())
            long_eternal = int(voting[4].get_text())

            ####### Sillage #######
            sill_intimate = int(voting[5].get_text())
            sill_moderate = int(voting[6].get_text())
            sill_strong = int(voting[7].get_text())
            sill_enormus = int(voting[8].get_text())

            ####### Gender #######
            gender_female = int(voting[9].get_text())
            gender_more_fem = int(voting[10].get_text())
            gender_unisex = int(voting[11].get_text())
            gender_more_male = int(voting[12].get_text())
            gender_male = int(voting[13].get_text())

            ####### Price Value #######
            value_w_over = int(voting[14].get_text())
            value_over = int(voting[15].get_text())
            value_ok = int(voting[16].get_text())
            value_good = int(voting[17].get_text())
            value_great = int(voting[18].get_text())

            ####### CREATING THE DICTIONARY OF DATA #######
            perfume_dict = {"name": perfume_name,
                            "company": perfume_comp,
                            "image": perfume_image,
                            "for_gender": for_gender,
                            "rating": rating,
                            "number_votes": number_votes,
                            "main accords": accords_dict,
                            "description": description,
                            "top notes": top_notes_list,
                            "middle notes": middle_notes_list,
                            "base notes": base_notes_list,
                            "longevity":   {"very weak": long_v_weak,
                                            "weak": long_weak,
                                            "moderate": long_moderate,
                                            "long lasting": long_long_last,
                                            "eternal": long_eternal},
                            "sillage":     {"intimate": sill_intimate,
                                            "moderate": sill_moderate,
                                            "strong": sill_strong,
                                            "enormous": sill_enormus},
                            "gender_vote": {"female": gender_female,
                                            "more female": gender_more_fem,
                                            "unisex": gender_unisex,
                                            "more male": gender_more_male,
                                            "male": gender_male},
                            "price value": {"way overpriced": value_w_over,
                                            "overpriced": value_over,
                                            "ok": value_ok,
                                            "good value": value_good,
                                            "great value": value_great}
                           }
            list_of_perfume_dicts.append(perfume_dict)

            time.sleep(2)

            browser.back()
            
            time.sleep(2)
        
#     except:
#         print(f"Error with {i} year - skipping to next year")
#         print(f"-------------------------------------------")

IndexError: list index out of range

In [26]:
len(list_of_perfume_dicts)

70

In [27]:
list_of_perfume_dicts

[{'name': 'Agua Lavanda',
  'company': 'Antonio Puig',
  'image': 'https://fimgs.net/mdimg/perfume/375x500.18411.jpg',
  'for_gender': 'for men',
  'rating': 4.29,
  'number_votes': 137,
  'main accords': {'fresh spicy': 100.0,
   'aromatic': 96.8902,
   'lavender': 88.2391,
   'woody': 62.0032,
   'citrus': 50.9208,
   'mossy': 48.6591,
   'herbal': 48.5743},
  'description': 'Agua Lavanda by Antonio Puig is a Aromatic Fougere fragrance for men. Agua Lavanda was launched in 1940. Top notes are Lavender, Rosemary and Bergamot; middle notes are Geranium and Nutmeg; base notes are Oakmoss, Cedar, Musk and Tonka Bean.  Read about this perfume in other languages: Deutsch, Español, Français, Italiano, Русский, Polski, Português, Ελληνικά, 汉语, Nederlands, Srpski, Română, العربية, Українська, Монгол, עברית.',
  'top notes': ['Lavender', 'Rosemary', 'Bergamot'],
  'middle notes': ['Geranium', 'Nutmeg'],
  'base notes': ['Oakmoss', 'Cedar', 'Musk', 'Tonka Bean'],
  'longevity': {'very weak': 14

In [28]:
with open ("perfume_data_JV_1940s.json", "w") as f:
    json.dump(list_of_perfume_dicts, f)

In [ ]:
####################################################
####### USE THIS FOR A SINGULAR PERFUME PAGE #######
############### NOT THE SEARCH PAGE ###############
###################################################


html = browser.html
soup = bs(html, "html.parser")

time.sleep(1)

####### PULL FROM ORIGINAL NAME LIST #######
perfume_name = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()
perfume_comp = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()

perfume_image = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]

for_gender = soup.find("small").get_text()

try:
    rating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
    number_votes = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
except:
    rating = "NA"
    number_votes = "NA"
    print(f"{perfume_name} does not have a ranking")

try:
    description = soup.find_all("div", class_="cell small-12")[3].get_text()
except:
    description = "NA"
    print(f"{perfume_name} does not have a description")

####### MAIN ACCORDS DICTIONARY #######

try:
    main_accords = soup.find_all("div", class_="cell accord-box")
    accords_dict = {}
    for m in range(len(main_accords)):
        accord_name = main_accords[m].get_text()
        accord_value = float(main_accords[m].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
        accords_dict[accord_name] = accord_value
except:
    accords_dict = {}
    print(f"{perfume_name} does not have accords")

####### FRAGRANCE NOTES #######        
notes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: row wrap; align-items: flex-end; padding: 0.5rem;"})

if len(notes) == 3:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for n in range(len(notes[0].find_all("span", class_="link-span"))):
        top_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3

    number = 2
    for p in range(len(notes[1].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[1].find_all("div")[number].get_text())
        number += 3

    number = 2
    for q in range(len(notes[2].find_all("span", class_="link-span"))):
        base_notes_list.append(notes[2].find_all("div")[number].get_text())
        number += 3
elif len(notes) == 2:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for r in range(len(notes[0].find_all("span", class_="link-span"))):
        top_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3

    number = 2
    for s in range(len(notes[1].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[1].find_all("div")[number].get_text())
        number += 3
elif len(notes) == 1:
    number = 2
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

    for v in range(len(notes[0].find_all("span", class_="link-span"))):
        middle_notes_list.append(notes[0].find_all("div")[number].get_text())
        number += 3
else:
    top_notes_list = []
    middle_notes_list = []
    base_notes_list = []

####### VOTING DATA & INFORMATION #######
voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")

####### Longevity #######
long_v_weak = int(voting[0].get_text())
long_weak = int(voting[1].get_text())
long_moderate = int(voting[2].get_text())
long_long_last = int(voting[3].get_text())
long_eternal = int(voting[4].get_text())

####### Sillage #######
sill_intimate = int(voting[5].get_text())
sill_moderate = int(voting[6].get_text())
sill_strong = int(voting[7].get_text())
sill_enormus = int(voting[8].get_text())

####### Gender #######
gender_female = int(voting[9].get_text())
gender_more_fem = int(voting[10].get_text())
gender_unisex = int(voting[11].get_text())
gender_more_male = int(voting[12].get_text())
gender_male = int(voting[13].get_text())

####### Price Value #######
value_w_over = int(voting[14].get_text())
value_over = int(voting[15].get_text())
value_ok = int(voting[16].get_text())
value_good = int(voting[17].get_text())
value_great = int(voting[18].get_text())

####### CREATING THE DICTIONARY OF DATA #######
perfume_dict = {"name": perfume_name,
                "company": perfume_comp,
                "image": perfume_image,
                "for_gender": for_gender,
                "rating": rating,
                "number_votes": number_votes,
                "main accords": accords_dict,
                "description": description,
                "top notes": top_notes_list,
                "middle notes": middle_notes_list,
                "base notes": base_notes_list,
                "longevity":   {"very weak": long_v_weak,
                                "weak": long_weak,
                                "moderate": long_moderate,
                                "long lasting": long_long_last,
                                "eternal": long_eternal},
                "sillage":     {"intimate": sill_intimate,
                                "moderate": sill_moderate,
                                "strong": sill_strong,
                                "enormous": sill_enormus},
                "gender_vote": {"female": gender_female,
                                "more female": gender_more_fem,
                                "unisex": gender_unisex,
                                "more male": gender_more_male,
                                "male": gender_male},
                "price value": {"way overpriced": value_w_over,
                                "overpriced": value_over,
                                "ok": value_ok,
                                "good value": value_good,
                                "great value": value_great}
               }
list_of_perfume_dicts.append(perfume_dict)

time.sleep(2)

browser.back()